In [1]:
# get all the data from the site https://www.epfl.ch/campus/associations/list/

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


url = 'https://www.epfl.ch/campus/associations/list/'

response = requests.get(url, headers=headers)
print(f'Status code: {response.status_code}')
print(f'Content type: {response.headers["content-type"]}')
#print(f'text: {response.text}')
soup = BeautifulSoup(response.text, 'html.parser')

# get the table

tables = soup.find_all('ul')
wanted_table = tables[2]


# get the rows
items = wanted_table.find_all('li')

# get the data
#when possible for each we want url and name and fullname
#most look like this -> <li><strong><a href="http://adec.epfl.ch/" rel="noopener noreferrer" target="_blank">ADEC</a></strong> (Association of EPFL Students in Chemistry)</li>
# some don't have a link -> <li><strong>AGEPoly</strong> (General Association of EPFL Students)</li>
#and some are wrapped in a span -> <li><span><strong><a href="http://www.aiesec.ch/epfl" rel="noopener noreferrer" target="_blank">AIESEC</a></strong> (International Association of Students in Economic and Commercial Sciences)</span></li>
data = {}
for item in items:
    # Find the first <a> tag
    link = item.find('a')
    text = item.text.strip()[:-1]
    parts = text.split('(')
    acronym = parts[0].strip()
    try:
        fullname = parts[1].strip()
    except IndexError:
        fullname = acronym
    # Extract acronym, URL, and full name if the <a> tag exists
    if link:
        url = link.get('href', 'N/A')
    else:
        # If <a> tag does not exist, extract text and assume it contains both acronym and full name
        url = 'N/A'

    data[acronym] = {'url': url, 'fullname': fullname}
    # Extract full name

# write to csv
with open('epfl_associations.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['acronym', 'url', 'fullname'])
    for acronym, info in data.items():
        writer.writerow([acronym, info['url'], info['fullname']])







Status code: 200
Content type: text/html; charset=UTF-8


In [2]:
#transform to panda into a a json
df = pd.read_csv('epfl_associations.csv')
df.to_json('epfl_associations.json', orient='records')



In [1]:
# To get started, simply uncomment the below code or create your own.
# Deploy with `firebase deploy`

from firebase_functions import https_fn
from firebase_admin import initialize_app, db


from firebase_functions import https_fn, options
import epflpeople
from firebase_functions.firestore_fn import (
    on_document_written,
    Event,
    Change,
    DocumentSnapshot,
)
from flask import jsonify
from google.cloud import firestore
firestore_client: google.cloud.firestore.Client = firestore.Client()
initialize_app()

@https_fn.on_call()
def oncallFind(req: https_fn.Request) -> https_fn.Response:
    # the request body is a JSON object with a single key "email"
    email = req.data["email"]
    userID = req.auth.uid
    profile = epflpeople.find(email)

    # profile is a format of a list of dictionnary we want first elem and change the dict into a json
    if profile:
        # {'name': 'Philippe', 'firstname': 'Marin Marc Alain'
        # , 'email': 'marin.philippe@epfl.ch'
        # , 'sciper': '326343'
        # , 'rank': 0
        # , 'accreds':
        #       [{'acronym': 'IN-MA2',
        #       'path': 'EPFL/ETU/IN-S/IN-MA2',
        #       'name': "Section d'informatique - Master semestre 2",
        #       'position': 'Etudiant',
        #       'phoneList': [],
        #       'officeList': [],
        #       'rank': 0, 'code': 50038}],
        # 'profile': 'marin.philippe'}
        associations = list()
        print(profile[0])

        for i in range(len(profile[0]["accreds"]) - 1):
            temp = {
                "acronym": profile[0]["accreds"][i + 1]["acronym"],
                "position": profile[0]["accreds"][i + 1]["position"],
                "rank": profile[0]["accreds"][i + 1]["rank"],
            }
            associations.append(temp)
        user = {
            "email": profile[0]["email"],
            "name": profile[0]["name"],
            "firstname": profile[0]["firstname"],
            "sciper": profile[0]["sciper"],
            "semester": profile[0]["accreds"][0]["acronym"],
            "associations" : associations
        }
        
        emailHash = hash(email)

        firestore_client.document(f"users/{userID}").set(user)
         # transform profile insto a json format
         
         
         
        return https_fn.Response(jsonify(profile[0]))
    else:
        return https_fn.Response("No profile found for email: " + email, status=404)


In [2]:
import epflpeople


profile = epflpeople.find("amaury.george@epfl.ch")

print(profile)

[{'name': 'George', 'firstname': 'Amaury Olivier Marie-Josèphe', 'email': 'amaury.george@epfl.ch', 'sciper': '324402', 'rank': 0, 'accreds': [{'acronym': 'SC-MA2', 'path': 'EPFL/ETU/SC-S/SC-MA2', 'name': 'Section de systèmes de communication - Master semestre 2', 'position': 'Etudiant', 'phoneList': [], 'officeList': [], 'rank': 0, 'code': 50110}], 'profile': 'amaury.george'}]


In [38]:
acrList = find_original_acronym("ROCKETTEAM")

string = ""

for i in acrList:
    string += i + " "

#remove the last space
string = string[:-1]
print (string)

EPFL ROCKET TEAM


In [39]:
acron = " ".join(acrList)
print(acron)


EPFL ROCKET TEAM


In [53]:
from firebase_functions import https_fn
from firebase_admin import initialize_app, db


from firebase_functions import https_fn, options
import epflpeople
from firebase_functions.firestore_fn import (
    on_document_written,
    Event,
    Change,
    DocumentSnapshot,
)
from flask import jsonify
from google.cloud import firestore
import os
import firebase_admin
from firebase_admin import credentials

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./the-software-enterprise-c5ea2-firebase-adminsdk-yconf-ba9fbdaa33.json"

firestore_client = firestore.Client()
#get print all associations

all_associations = firestore_client.collection("associations").stream()

for association in all_associations:
    #for all associations add a field that is there acronym in lowercase
    association_dict = association.to_dict()
    firestore_client.collection("associations").document(association.id).update({"acronym_lower":association_dict["acronym"].lower()})
    
    
    
  
    

